# Preprocessing

We preprocess the data twice: Once without imputing values, because this can be handled by the random forest models, and once with imputing values, because this is necessary for the MLP model.

This file is self-contained to maximize readability and compatibility.

# Setup

## Paths

In [1]:
excel_path = './data/anonymized_data_dsl_project_5.xlsx'
save_data_path = './processed_data'

## Importing necessary libraries

In [2]:
import pandas as pd
import numpy as np
from os import path

## Functions for data imputation

In [3]:
def drop_sparse_col(df, columns, threshold=0.5):
    for col in columns:
        if df[col].isnull().sum() / len(df) > threshold:
            print(f'Dropping column {col} with {df[col].isnull().sum() / len(df) * 100:.2f}% missing values')
            df.drop(col, axis=1, inplace=True)
        else:
            print(f'Keeping column {col} with {df[col].isnull().sum() / len(df) * 100:.2f}% missing values')
    print('###############################################')
    return

def fill_na(df, columns, how):
    if how == 'dominant':
        for col in columns:
            if col in df.columns:  # Check if the column exists in the DataFrame
                value = df[col].mode()[0]  # Get the dominant value (mode)
                dominant_class_frequency = df[col].value_counts().max()  # Get the frequency of the dominant class
                missing_values = df[col].isnull().sum()  # Get the number of missing values in the column
                print(f"Column: {col}, Dominant Class: {value}, Frequency: {dominant_class_frequency}, Missing Values: {missing_values}")
                df[col] = df[col].fillna(value)  # Fill missing values with the dominant class
            else:
                pass
    elif how == 'zero':
        for col in columns:
            if col in df.columns:
                print(f'Number of missing values in {col}: {df[col].isnull().sum()}, (filled with 0)')
                df[col] = df[col].fillna(0)
            else:
                pass
    elif how == 'mean':
        for col in columns:
            if col in df.columns:
                print(f'Number of missing values in {col}: {df[col].isnull().sum()}, (filled with mean)')
                df[col] = df[col].fillna(df[col].mean())
            else:
                pass
    else:
        print('Please specify the method to fill missing values')
    print('###############################################')
    return

def drop_na(df, columns):
    for col in columns:
        if col in df.columns:
            print(f'Number of missing values in {col}: {df[col].isnull().sum()}, (dropped)')
        else:
            print(f"Column '{col}' not found in the DataFrame.")

    # Drop rows with NA values in any of the specified columns, after the loop
    df = df.dropna(subset=[col for col in columns if col in df.columns])

    print('###############################################')
    return df


def one_hot_encode(df, column_name, map):
    df.loc[:, column_name] = df[column_name].map(map)
    print(f'Number of Nan values in {column_name}: {df[column_name].isnull().sum()}')
    df = pd.get_dummies(df, columns=[column_name], dummy_na=True)
    print(f'One-hot encoded column: {column_name}')
    print('###############################################')
    return df

def convert_bool_to_int(df):
    bool_columns = [col for col in df.columns if df[col].dtype == 'bool']
    df[bool_columns] = df[bool_columns].astype(int)
    return df

## Mappings from given data representation to numerical values

In [4]:
renamed_columns = {
    'nu-desc_score_total_?': 'nu-desc_score_total_BI',
    '14_anxiety _nrs_0-10_BI': '14_anxiety_nrs_BI',
    '15:_stress_nrs _0-10_BI': '15_stress_nrs_BI',
    '18b:_vomiting_nrs _0-10_BI': '18b_vomiting_nrs_BI',
    '3:_asa-ps\n': '3:_asa-ps',
}


columns_to_drop = [ 'site',
          'baseline_no',
          'question_116',
          'appointment_id',
          'schema_version_id',
          'start_date',
          '0._date:',
          '8:_oral_consent',
          '30: suppression_time_total',
          '29:_satisfaction_with_BI',
          '30: minutes_of_anaesthesia_start_delay_(0_-_240min)',
          '29:_satisfaction_with_treatment_premedication_talk_AP',
          'sbi_team_handover_performed_arrival_pacu',
          '29:_satisfaction_with_treatment_premedication_talk_DP',
          'sbi_team_handover_performed_discharge_pacu',
          '9:_pain_BI', #summary column
          '9:_pain_DP', #summary column
          '9:_pain_HV', #summary column
          '9:_pain_AP', #summary column
          '5:_setting', #summary column
          '6:_anaesthesia', #summary column
          '7:_prevention', #summary column
          'nu-desc_score_total_AP',#summary column
          'nu-desc_score_total_DP',#summary column
          'nu-desc_score_total_HV',#summary column
          'nu-desc_score_total_BI',#summary column
          '27:_laryngeal_mask_BI', #summary column
          ]

post_op_map = {
    0: 'PACU',
    #1: 'ICU', #not in dataset
    2: 'WARD',
    3: 'IMCU' #renamed from Intermediate Care Unit
}

surigcal_specialty_map = {
    '0': 'General',
    '1': 'Trauma/Orth.',
    '2': 'Urology',
    '3': 'Gyn/Obs',
    '4': 'Ophtalmic',
    '5': 'Plastic/breast',
    '6': 'Otolaryngology',
    '7': 'Thoracic',
    '8': 'Neuro',
    '9': 'Cardiac',
    'a': 'Oral/maxilofacial',
    'b': 'Vascular',
    'c': 'Other'
}


laryngeal_map = {
    0: 'LMA',
    1: 'Tube',
    2: 'Muscle relaxants',
    3: 'Video laryngoscope',
    4: 'Other'
}


## Loading the Excel file

In [5]:
df = pd.read_excel(excel_path)

# Processing the data

In [6]:
df.columns = df.columns.str.replace('\xa0', ' ', regex=True)
df.rename(columns=renamed_columns, inplace=True)
df.drop(columns_to_drop, axis=1, inplace=True)

df['Gender'] = df['Gender'].apply(lambda x: 1 if x == 'M' else (0 if x == 'F' else print("unknown gender")))

pain_columns = [col for col in df.columns if 'pain' in col.lower() and
                any(substring in col for substring in ['DP', 'HV', 'AP', 'BI'])]
drop_sparse_col(df, pain_columns, threshold=0.5)#0 is dominant and neutral

df = one_hot_encode(df, '2:_post-op', post_op_map)

df = one_hot_encode(df, '4:_surgical_speciality', surigcal_specialty_map)

setting_columns = [col for col in df.columns if 'setting' in col.lower()]

anaesthesia_columns = [col for col in df.columns if '_anaesthesia' in col.lower()]

prevention_columns = [col for col in df.columns if '_prevention' in col.lower()]

rass_columns = [col for col in df.columns if '_rass' in col.lower()]

nu_desc_substr = ['_disorientation', '_inappropriate_behavior',
              '_inappropriate_communication',
              'illusions/hallucinations', '_psychomotor_retardation']
nu_desc_cols = [col for col in df.columns if any(sub in col.lower() for sub in nu_desc_substr)]

moyb_columns = [col for col in df.columns if '_moyb_' in col.lower()]

anxiety_columns = [col for col in df.columns if 'anxiety' in col.lower()]

stress_columns = [col for col in df.columns if '_stress' in col.lower()]

well_being_columns = [col for col in df.columns if 'well-being' in col.lower()]

nausea_columns = [col for col in df.columns if 'nausea' in col.lower()]

vomit_columns = [col for col in df.columns if 'vomit' in col.lower()]

substrings = ['visual_aid', 'hearing_aid',
              'dentures']
aids_colums = [col for col in df.columns if any(sub in col.lower() for sub in substrings)]
fill_na(df, aids_colums, how='zero') #assume missing data = no = 0

fluid_columns = [col for col in df.columns if 'fluid' in col.lower()]

thirst_columns = [col for col in df.columns if 'thirst' in col.lower()]

temperature_columns = [col for col in df.columns if 'temperature' in col.lower()]

noise_columns = [col for col in df.columns if 'noise' in col.lower()]

shivering_columns = [col for col in df.columns if 'shivering' in col.lower()]

laryngeal_columns = [col for col in df.columns if 'laryngeal' in col.lower()]
laryngeal_binary_columns = [col for col in laryngeal_columns if df[col].isin([1, np.nan]).all()]
laryngeal_discomfort_columns = [col for col in laryngeal_columns if col not in laryngeal_binary_columns]
fill_na(df, laryngeal_discomfort_columns, how='dominant') # missing data means no discomfort

bladder_columns = [col for col in df.columns if 'bladder' in col.lower()]

quality_columns = [col for col in df.columns if 'quality' in col.lower()]

length_columns = [col for col in df.columns if 'length_of_patient_stay' in col.lower()]


Keeping column 9:_pain_BI_Resting_NRS with 0.21% missing values
Dropping column 9:_pain_BI_Resting_BPS with 99.51% missing values
Keeping column 9:_pain_BI_Mobilization_NRS with 0.91% missing values
Dropping column 9:_pain_BI_Mobilization BPS with 99.65% missing values
Keeping column 9:_pain_AP_Resting_NRS with 6.32% missing values
Dropping column 9:_pain_AP_Resting_BPS with 94.81% missing values
Keeping column 9:_pain_AP_Mobilization_NRS with 7.30% missing values
Dropping column 9:_pain_AP_Mobilization BPS with 94.95% missing values
Keeping column 9:_pain_DP_Resting_NRS with 0.70% missing values
Dropping column 9:_pain_DP_Resting_BPS with 99.86% missing values
Keeping column 9:_pain_DP_Mobilization_NRS with 2.88% missing values
Dropping column 9:_pain_DP_Mobilization BPS with 99.86% missing values
Keeping column 9:_pain_HV_Resting_NRS with 0.70% missing values
Dropping column 9:_pain_HV_Resting_BPS with 95.72% missing values
Keeping column 9:_pain_HV_Mobilization_NRS with 1.33% missin

/var/folders/kg/82lcvppx34n625kn45bnl_980000gn/T/ipykernel_93292/1232898196.py:56: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['PACU' 'PACU' 'PACU' ... 'PACU' 'IMCU' 'PACU']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, column_name] = df[column_name].map(map)


In [7]:
df = convert_bool_to_int(df)
# If a column has only 1s and NaNs, we can assume that the missing values are 0
# Find columns where this is the case, but if it is completely empty, we can't assume anything and should drop it
cols = df.columns
dropped_cols = []
for col in cols:
    if df[col].isna().sum() == len(df):
        df.drop(col, axis=1, inplace=True)
        dropped_cols.append(col)
    elif df[col].isna().sum() + df[col].sum() == len(df):
        df[col] = df[col].fillna(0)
print(f"Dropped columns: {dropped_cols} because they were completely empty")
# print all column names value counts and nan counts
counter = 0
nan_count_dict = {}
for column in df.columns:
    # Only print columns with NaN values
    if df[column].isna().sum() != 0:
        print(f"-----------{counter}-----------")
        print(f"\nValue counts for {column}:")
        print(df[column].value_counts())
        print(f"Number of NaN values for {column}: {df[column].isna().sum()}")
        nan_count_dict[column] = df[column].isna().sum()
        counter += 1
# print sorted nan counts
print(f"Sorted nan counts:")
sorted_nan_count_dict = dict(sorted(nan_count_dict.items(), key=lambda item: item[1]))
# Pretty print
for key, value in sorted_nan_count_dict.items():
    print(f"{key}: {value}")
# Save unprocessed df with NaN values into other variable
df_unprocessed = df.copy()
df_unprocessed.to_csv(path.join(save_data_path, 'df_not_imputed.csv'), index=False)

Dropped columns: ['(5:_setting)_Other_intervention', '6:_anaesthesia_Regional anaesthesia (nerve block; e.g., only blocking arm nerve for hand surgery. Not on brain, so should give less cognitive decline)', '6:_anaesthesia_Thorax and abdominal wall', '6:_anaesthesia_Head and Neck', '7:_prevention_PACU', '7:_prevention_OR'] because they were completely empty
-----------0-----------

Value counts for 3:_asa-ps:
3:_asa-ps
1.0    677
2.0    418
0.0    231
3.0      7
Name: count, dtype: int64
Number of NaN values for 3:_asa-ps: 92
-----------1-----------

Value counts for 9:_pain_BI_Resting_NRS:
9:_pain_BI_Resting_NRS
0.0     1001
2.0      105
3.0       76
5.0       59
4.0       51
1.0       48
6.0       36
7.0       25
8.0       17
10.0       2
9.0        2
Name: count, dtype: int64
Number of NaN values for 9:_pain_BI_Resting_NRS: 3
-----------2-----------

Value counts for 9:_pain_BI_Mobilization_NRS:
9:_pain_BI_Mobilization_NRS
0.0     731
2.0      99
8.0      88
5.0      86
3.0      82


## Imputation using simple heuristics

In [8]:
fill_na(df, pain_columns, how='dominant') 
fill_na(df, ['3:_asa-ps'], how='dominant')
fill_na(df, setting_columns, how='zero') #assume missing missing data = no = 0
fill_na(df, anaesthesia_columns, how='zero') #assume missing missing data = no = 0
fill_na(df, prevention_columns, how='zero') #assume missing data = no = 0
fill_na(df, rass_columns, how='zero') #assume neutral more likely not to be registered
fill_na(df, nu_desc_cols, how='dominant')
fill_na(df, moyb_columns, how='zero') #assume missing data = no = 0
fill_na(df, anxiety_columns, how='zero') #assume missing data = no = 0
fill_na(df, stress_columns, how='zero') #assume missing data = no = 0
fill_na(df, well_being_columns, how='dominant') #assume missing data = unremarkable
fill_na(df, nausea_columns, how='zero') #assume missing data = no = 0
fill_na(df, vomit_columns, how='zero') #assume missing data = no = 0
fill_na(df, aids_colums, how='zero') #assume missing data = no = 0
fill_na(df, [fluid_columns[0]], how='mean')  #before induction, no other obvisous choice
fill_na(df, [fluid_columns[1]], how='zero')  #pacu, assume missing data = no = 0
fill_na(df, thirst_columns, how='mean') #no clear dominant choice
fill_na(df, temperature_columns, how='dominant') #no measurement = no fever
fill_na(df, noise_columns, how='zero') #assume missing data = no = 0
fill_na(df, shivering_columns, how='dominant') #assume missing data = no = 0
fill_na(df, bladder_columns, how='zero') #assume missing data = no = 0
fill_na(df, quality_columns, how='dominant') #assume missing data = unremarkable
fill_na(df, length_columns, how='mean') #no clear dominant choice
# fill_na(df, ["26:_surgical_time_BI"], how='mean') #no clear dominant choice
df.to_csv(path.join(save_data_path, 'df_imputed.csv'), index=False)

Column: 9:_pain_BI_Resting_NRS, Dominant Class: 0.0, Frequency: 1001, Missing Values: 3
Column: 9:_pain_BI_Mobilization_NRS, Dominant Class: 0.0, Frequency: 731, Missing Values: 13
Column: 9:_pain_AP_Resting_NRS, Dominant Class: 0.0, Frequency: 965, Missing Values: 90
Column: 9:_pain_AP_Mobilization_NRS, Dominant Class: 0.0, Frequency: 929, Missing Values: 104
Column: 9:_pain_DP_Resting_NRS, Dominant Class: 0.0, Frequency: 741, Missing Values: 10
Column: 9:_pain_DP_Mobilization_NRS, Dominant Class: 0.0, Frequency: 685, Missing Values: 41
Column: 9:_pain_HV_Resting_NRS, Dominant Class: 0.0, Frequency: 695, Missing Values: 10
Column: 9:_pain_HV_Mobilization_NRS, Dominant Class: 0.0, Frequency: 660, Missing Values: 19
###############################################
Column: 3:_asa-ps, Dominant Class: 1.0, Frequency: 677, Missing Values: 92
###############################################
Number of missing values in (5:_setting)_Elective: 0, (filled with 0)
Number of missing values in (5:_se

## Imputation using Imputers

In [9]:
# We use complex imputation for columns that have a lot of missing values
df = df_unprocessed.copy()
# Remove target columns before imputing to avoid data leakage
target_substrings = ['_disorientation', '_inappropriate_behavior',
              '_inappropriate_communication',
              'illusions/hallucinations', '_psychomotor_retardation']
target_cols = [col for col in df.columns if any(sub in col.lower() for sub in target_substrings)]
# target cols do not contain BI cols
target_cols = [col for col in target_cols if 'BI' not in col]
# Save the target columns for later
df_targets = df[target_cols]
df.drop(target_cols, axis=1, inplace=True)
# Impute the rest of the columns using KNN imputer
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
# Add the target columns back
df_imputed = pd.concat([df_imputed, df_targets], axis=1)
df_imputed.to_csv(path.join(save_data_path, 'df_imputed_complex.csv'), index=False)
# Check if there are any NaN values left

